In [ ]:
import json
import ctypes
from dask.distributed import Client

def trim_memory() -> int:
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)

In [ ]:
import numpy as np
import pandas as pd
import dask
import dask.dataframe as dd
from dask.distributed import Client
import ctypes
import time

In [ ]:
client = Client()
client.run(trim_memory)
client = client.restart()

In [1]:
client

NameError: name 'Client' is not defined

In [4]:
user_reviews_ddf = dd.read_csv("user_reviews_Release.csv")
#user_reviews_ddf.visualize()
user_reviews_ddf

In [7]:
user_reviews_ddf.head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
npartitions=445,,,,,,,,,
,object,object,object,object,object,float64,object,float64,object
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [ ]:
num_partitions = user_reviews_ddf.npartitions
print(f"Total number of partitions = {num_partitions}")

number_products_rated = user_reviews_ddf.groupby("reviewerID", sort = False).asin.count(split_out=num_partitions)
#number_products_rated.compute()

In [ ]:
avg_ratings = user_reviews_ddf.groupby("reviewerID", sort=False).overall.mean(split_out=100)
#avg_ratings.compute()

In [ ]:
year = user_reviews_ddf.map_partitions(lambda df: df.reviewTime.apply(lambda x: int(x[-4:])), meta=pd.Series(dtype=int))
user_reviews_ddf['year'] = year
reviewing_since = user_reviews_ddf.groupby('reviewerID',sort=False).year.min(split_out=100)
#reviewing_since.compute()

In [ ]:
user_reviews_ddf['helpful_votes']  = user_reviews_ddf.map_partitions(lambda df: df.helpful.apply(lambda x: x[1:-1].split(',')[0]).astype(int), meta=pd.Series(dtype=int))
user_reviews_ddf['total_votes'] = user_reviews_ddf.map_partitions(lambda df: df.helpful.apply(lambda x: x[1:-1].split(',')[1]), meta=pd.Series(dtype=int))

#user_reviews_ddf['helpful_votes'].describe()

In [ ]:
helpful_votes = user_reviews_ddf.groupby('reviewerID', sort=False).helpful_votes.sum(split_out=100)
total_votes = user_reviews_ddf.groupby('reviewerID',sort=False).total_votes.sum(split_out=100)
#helpful_votes.compute()

In [ ]:
import time
start_time = time.time()
dd_res = dd.concat([number_products_rated, avg_ratings, reviewing_since, helpful_votes, total_votes], axis=1)
dd_res = dd_res.rename(columns={'asin': 'number_products_rated', 'overall': 'avg_ratings', 'year':reviewing_since})
submit = dd_res.describe().compute().round(2)
with open('results_PA0.json', 'w') as outfile:
        json.dump(json.loads(submit.to_json()), outfile)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.6f} seconds")

In [ ]:
#dd_res.tail(10)

In [ ]:
import time

client = Client()
client.run(trim_memory)
client.restart()

In [ ]:

start_time = time.time()

def PA0(user_reviews_csv_path):
#     client = Client()
#     client.run(trim_memory)
#     client.restart()
    user_reviews_ddf = dd.read_csv(user_reviews_csv_path)
    number_products_rated = user_reviews_ddf.groupby("reviewerID", sort = False).asin.count(split_out=100) 
    avg_ratings = user_reviews_ddf.groupby("reviewerID", sort=False).overall.mean(split_out=100)
    user_reviews_ddf['year'] = user_reviews_ddf.map_partitions(lambda df: df.reviewTime.apply(lambda x: int(x[-4:])), meta=pd.Series(dtype=int))
    reviewing_since = user_reviews_ddf.groupby('reviewerID',sort=False).year.min(split_out=100)

    user_reviews_ddf['helpful_votes']  = user_reviews_ddf.map_partitions(lambda df: df.helpful.apply(lambda x: x[1:-1].split(',')[0]).astype(int), meta=pd.Series(dtype=int))
    user_reviews_ddf['total_votes'] = user_reviews_ddf.map_partitions(lambda df: df.helpful.apply(lambda x: x[1:-1].split(',')[1]).astype(int), meta=pd.Series(dtype=int))

    helpful_votes = user_reviews_ddf.groupby('reviewerID', sort=False).helpful_votes.sum(split_out=100)
    total_votes = user_reviews_ddf.groupby('reviewerID',sort=False).total_votes.sum(split_out=100)

    dd_res = dd.concat([number_products_rated, avg_ratings, reviewing_since, helpful_votes, total_votes], axis=1)
    dd_res = dd_res.rename(columns={'asin': 'number_products_rated', 'overall': 'avg_ratings', 'year': 'reviewing_since'})
    submit = dd_res.describe().compute().round(2)
    with open('results_PA0.json', 'w') as outfile:
        json.dump(json.loads(submit.to_json()), outfile)
    return submit

submit = PA0("user_reviews_Release.csv")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.6f} seconds")
